In [16]:

# Data manipulation
import pandas as pd
import numpy as np
import re

# Load dynamic exchange rates module (no automatic fetch here)
import sys
import os
notebook_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if notebook_root not in sys.path:
    sys.path.insert(0, notebook_root)

try:
    from scripts.exchange_rates import fetch_latest_rates
    print('exchange_rates module imported')
except Exception as e:
    print('Failed to import exchange_rates, falling back to static fetch:', e)
    def fetch_latest_rates(symbols=None, use_cache=True):
        return {'EUR':1.0,'USD':0.8779,'GBP':1.1870,'JPY':0.006076,'CHF':1.0660}


exchange_rates module imported


In [17]:
#clean up the data to silver level

def clean_excel_file(file_path, output_path=None):
    """
    Load, clean, and optionally save cleaned Excel data:
    - Removes rows with Elferspot Shop links
    """
    df = pd.read_excel(file_path)
    df = df.drop_duplicates(subset=['URL'])  # or based on ['Title', 'Year', 'Mileage_km']
    df = df.drop(columns=['License documents (Click to open)'])
    
    # Filter out unwanted shop links
    df = df[~df["URL"].str.contains("https://www.elferspot.com/en/shop/", na=False)]
    df[["mileage_value", "mileage_unit"]] = df["Mileage"].str.extract(r'([\d,]+)\s*(km|mi)', expand=True)
    # Fill missing mileage_value with 1 and mileage_unit with 'km'
    # Only fill missing mileage_value with "1" if Condition is "Fully restored"
    mask = (df["mileage_value"].isna()) & (df["Condition"].str.contains("fully restored", case=False, na=False))
    df.loc[mask, "mileage_value"] = "1"
    df = df[~df["mileage_value"].isna()]
    df["mileage_unit"] = df["mileage_unit"].fillna("km").replace("", "km")
    df["Condition"] = df["Condition"].apply(lambda x: "Unknown" if pd.isna(x) or str(x).strip() == "" else x)
    df["Paint-to-Sample (PTS)"] = df["Paint-to-Sample (PTS)"].apply(lambda x: 1 if str(x).strip().lower() == "yes" else 0)
    df["Matching numbers"] = df["Matching numbers"].apply(lambda x: "Unknown" if pd.isna(x) or str(x).strip() == "" else x)
    df["Number of vehicle owners"] = df["Number of vehicle owners"].apply(lambda x: "Unknown" if pd.isna(x) or str(x).strip() == "" else x)
    df['owners_known'] = df['Number of vehicle owners'].apply(lambda x: 0 if str(x).strip().lower() == "unknown" else 1)
    df['is_fully_restored'] = df['Condition'].str.contains("fully restored", case=False, na=False).astype(int)
    df['Series'] = df['Series'].replace('982 (718 Boxster/Cayman)', '982')
    # Remove commas and convert to float
    df["mileage_value"] = df["mileage_value"].str.replace(",", "").astype(float)
    
    # Convert miles to kilometers
    df["Mileage_km"] = np.where(
        df["mileage_unit"] == "mi",
        df["mileage_value"] * 1.60934,
        df["mileage_value"]
    )

    df["Mileage_km"] = df["Mileage_km"].round().astype("Int64")
    df["Mileage_sq"] = df["Mileage_km"] ** 2
    df['inv_mileage'] = 1 / (df['Mileage_km'] + 1) 
    
    # Extract currency and price from the Price column
    df[['currency', 'price']] = df['Price'].str.extract(r'([A-Za-z]+)\s([\d,]+)\s*$', expand=True)
    df['price'] = df['price'].str.replace(",", "").astype(float)

    # Calculate the lower and upper bounds for outliers
    # Q1 = df['Mileage_km'].quantile(0.25)
    # Q3 = df['Mileage_km'].quantile(0.75)
    # IQR = Q3 - Q1
    # lower_bound = Q1 - 1.5 * IQR
    # upper_bound = Q3 + 1.5 * IQR
    # df = df[(df['Mileage_km'] >= lower_bound) & (df['Mileage_km'] <= upper_bound)]
    df = df[(df['Mileage_km'] < 900000)]

    df['age'] = df['Year of construction'].apply(lambda x: pd.to_datetime("now").year - x).astype(int)
    df['mileage_per_year'] = df['Mileage_km'] / np.where(df['age'] == 0, 0.5, df['age'])
    df['log_mileage'] = np.log1p(df['Mileage_km'])  # Use log1p to handle zero mileage correctly    

    currency_rates = {
        'EUR': 1.0,
        'USD': 0.8779,  # 1 USD ≈ 0.8779 EUR
        'GBP': 1.1870,  # 1 GBP ≈ 1.1870 EUR
        'JPY': 0.006076,  # 1 JPY ≈ 0.006076 EUR
        'CHF': 1.0660,  # 1 CHF ≈ 1.0660 EUR
    }

    # Function to convert currency to EUR
    def convert_to_eur(amount, currency):
        conversion_rate = currency_rates.get(currency, 1.0)  # Default to 1.0 if currency not in list
        return amount * conversion_rate

    # Convert prices to EUR using the predefined conversion rates
    df['price_in_eur'] = np.vectorize(convert_to_eur)(df['price'], df['currency'])
    df = df[df["price_in_eur"] < 700000] # Filter out prices greater than 300,000
    df = df[df["price_in_eur"] > 15000]  

    # df = df[df["Mileage_km"] > 7000] # Filter out mileage greater than 300,000
    model_lower = df["Model"].str.lower().str.strip()

    # Define conditions for each category (grouped by similar models)
    conditions = [
        # Singer/Guntherwerks – high-end bespoke builds
        model_lower.str.contains(r"\b(singer|guntherwerks|gunther werks|Lanzante)\b"),

        # GT2RS – top tier track weapons and rarest models
        model_lower.str.contains(
            r"(gt2 rs|rsr|gt2 rsr|911 gt2 rs|sport classic|911 st\b|911 s[\s\/]?t|60 (jahre|years|anniversary)|911 r\b|le mans centenaire edition|991 club coupé|club coupé)",
            flags=re.IGNORECASE),     

        #GT3RS
        model_lower.str.contains(r"\b(gt3 rs|gt3rs|911 gt3 rs|ruf|dakar|gt2 clubsport)\b"),
        
        # RS Model – classic & special RS variants
        model_lower.str.contains(r"\b(964 carrera rs|993 carrera rs|carrera rs\b|911 carrera rs\b|rs america|911 carrera 2\.7|911 carrera 2,7|911 carrera 2\.7 rs|911 carrera 2\.7 mfi|flachbau|gt4 rs|gt4rs|leichtbau)\b"),

        # GT3 / GT2 – track-focused but not RS-level
        model_lower.str.contains(r"\b(gt3\b(?! rs)|gt2\b(?! rs)|911 gt3\b(?! rs)|911 gt2\b(?! rs)|cup|gt4|911 carrera 3\.2 clubsport)\b"),
        
        # Special / Backdate / Restomod – high-tier special editions or modifieds
        model_lower.str.contains(r"\b(speedster|clubsport|heritage|backdate|restomod|modified|exclusive manufaktur)\b"),
        
        # Turbo S / Turbo – forced induction across eras
        model_lower.str.contains(r"\b(turbo s\b|911 turbo s\b|turbo\b(?! rs|gt2|cup)|964 turbo|993 turbo|930 turbo|911 turbo 3\.3|911 turbo 3\.0|911 turbo|turbo wls|911 turbo cabriolet)\b"),
        
        # GTS – sporty trims across generations
        model_lower.str.contains(r"\b(gts|exclusive edition)\b"),
        
        # Carrera 3.0 / 3.2 / S / SC – air-cooled driver models
        model_lower.str.contains(r"\b(911 carrera 3\.0|carrera 3\.0|carrera 3\.2|911 carrera 3\.2|911 sc\b|911 s\b|carrera s\b|carrera 4s\b|993 carrera s\b|993 carrera 4s)\b"),
        
        # Base Carrera / Targa / 912 – entry-level and accessible classics
        model_lower.str.contains(r"\b(912\b|911\b|911 t\b|911 l\b|911 e\b|911 targa\b|carrera\b|carrera 2\b|cabriolet|targa|coupe|convertible)\b")
    ]

    choices = [
        "Bespoke / Rarest Models",
        "GT2RS and RARE Models",
        "GT3RS",
        "RS Model",
        "GT4 / GT3 / GT2",
        "Special / Backdate",
        "Turbo S / Turbo",
        "GTS",
        "Carrera 3.0/3.2 / S / SC",
        "Base Carrera / Targa / 912"
    ]

    # Default if no condition matches
    default = "GTS"

    # Create new column
    df["model_category"] = np.select(conditions, choices, default=default)
    df['model_cat_ordered'] = pd.Categorical(
        df['model_category'],
        categories=choices,
        ordered=True
    ).codes
    df['Mileage_model_cat'] = df['Mileage_km'] * df['model_cat_ordered']
    df['inv_Mileage_model_cat'] = df['inv_mileage'] * df['model_cat_ordered']
    df['Mileage_sq_model_cat'] = df['Mileage_sq'] * df['model_cat_ordered']


    df = df[~df["URL"].str.contains("https://www.elferspot.com/en/shop/", na=False)]
    df = df.drop(columns=['Error'])
    
    df['Ready to drive'] = df['Ready to drive'].fillna("")
    df['Ready to drive'] = df['Ready to drive'].replace(r'^\s*$', "no", regex=True)
    df.loc[(df['age'] < 30) & ((df['Ready to drive'] == 'No') | (df['Ready to drive'] == 'no')), 'Ready to drive'] = 'yes'

    
    df.reset_index(drop=True, inplace=True)

    # Optionally save cleaned file
    if output_path:
        df.to_excel(output_path, index=False)

    return df
    

clean_df = clean_excel_file("../data/all_listings_bronze.xlsx", output_path="../data/all_listings_silver.xlsx")
display(clean_df.head())

c:\Users\jaxon\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\function_base.py:2455: RuntimeWarning: invalid value encountered in convert_to_eur (vectorized)
  outputs = ufunc(*inputs)
C:\Users\jaxon\AppData\Local\Temp\ipykernel_21580\3575161825.py:83: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  model_lower.str.contains(r"\b(singer|guntherwerks|gunther werks|Lanzante)\b"),
C:\Users\jaxon\AppData\Local\Temp\ipykernel_21580\3575161825.py:86: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  model_lower.str.contains(
C:\Users\jaxon\AppData\Local\Temp\ipykernel_21580\3575161825.py:91: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  model_lower.str.contains(r"\b(gt3 rs|gt3rs|911 gt3 rs|ruf|dakar|gt2 clubsport)\b

,URL,level_0,index,Title,Year of construction,Model,VIN,Body,Series,Power,...,price,age,mileage_per_year,log_mileage,price_in_eur,model_category,model_cat_ordered,Mileage_model_cat,inv_Mileage_model_cat,Mileage_sq_model_cat
0,https://www.elferspot.com/en/car/porsche-912-1...,2.0,2.0,Porsche 912,1967.0,912,12804201,Coupé,912,118 HP,...,179800.0,58,15.517241,6.803505,179800.0,Base Carrera / Targa / 912,9,8100,0.009989,7290000
1,https://www.elferspot.com/en/car/porsche-911-t...,3.0,3.0,Porsche 911 T,1973.0,911 T (US),NaN,Coupé,911 F-Model,140 HP,...,82950.0,52,980.769231,10.839601,82950.0,Base Carrera / Targa / 912,9,459000,0.000176,23409000000
2,https://www.elferspot.com/en/car/porsche-964-c...,6.0,6.0,Porsche 964 Carrera 2,1992.0,964 Carrera 2,NaN,Cabriolet,964,250 HP,...,87500.0,33,4812.121212,11.975407,87500.0,Base Carrera / Targa / 912,9,1429200,0.000057,226956960000
3,https://www.elferspot.com/en/car/porsche-911-c...,7.0,7.0,Porsche 911 Carrera 3.2 WTL,1987.0,911 Carrera 3.2 WTL,WP0ZZZ91ZHS140870,Targa,911 G-Model,231 HP,...,159900.0,38,3347.710526,11.753626,159900.0,Carrera 3.0/3.2 / S / SC,8,1017704,0.000063,129465178952
4,https://www.elferspot.com/en/car/porsche-993-c...,8.0,8.0,Porsche 993 Targa,1997.0,993 Carrera,NaN,Targa,993,286 HP,...,110000.0,28,5265.928571,11.901224,110000.0,Base Carrera / Targa / 912,9,1327014,0.000061,195662906244


In [18]:
#feature extraction
def extract_listing_features(df, text_column='description'):
    """
    Extracts structured features from listing descriptions.

    Parameters:
    df (pd.DataFrame): DataFrame containing at least a `description` column.
    text_column (str): Column name for the listing description.

    Returns:
    pd.DataFrame: Input DataFrame with new binary feature columns and a score.
    """
    # Define keyword patterns
    patterns = {
    # FULL RESTORATION
    'restoration_full': r'\b('
        r'frame[- ]?off|'
        r'body[- ]?off|'
        r'complete restoration|'
        r'fully restored|'
        r'fully rebuilt|'
        r'nut and bolt (restoration|rebuild)|'
        r'completely restored|'
        r'full restioration|'
        r'ground[- ]?up restoration|'
        r'restored to (original|factory) specification|'
        r'mechanically restored|'
        r'interior refurbishment|'
        r'engine overhaul'
        r'category 1'
    r')\b',

    # PARTIAL RESTORATION
    'restoration_partial': r'\b('
        r'partial restoration|'
        r'cosmetic refresh|'
        r'lightly restored|'
        r'restored in parts|'
        r'category 2'
    r')\b',

    # RESTOMOD / CUSTOMIZED
    'is_restomod': r'\b('
        r'backdate|'
        r'restomod|'
        r'modified|'
        r'custom (build|interior|paint|exhaust|engine|body)'
    r')\b',

    # DOCUMENTATION / HISTORY
    'has_docs': r'\b('
        r'full (documentation|service history|records|history)|'
        r'extensive records|'
        r'well documented|'
        r'fully documented|'
    r')\b',

    # MATCHING NUMBERS
    'is_matching_numbers': r'\b('
        r'matching numbers|'
        r'numbers matching|'
        r'matching drivetrain'
    r')\b',

    # MINT / CONDITION
    'is_mint': r'\b('
        r'mint condition|'
        r'collector quality|'
        r'fully sorted|'
        r'excellent condition|'
        r'top condition|'
        r'showroom condition'
    r')\b',

    # RACE READY / TRACK PREP
    'is_race_ready': r'\b('
        r'rally ready|'
        r'race[- ]?ready|'
        r'track[- ]?prepped|'
        r'bucket seats|'
        r'racing harness|'
        r'homologated|'
        r'fire suppression system'
    r')\b',

    # RARE
    'is_rare': r'\b('
        r'rare\b|'
        r'special edition|'
        r'limited edition|'
        r'1 of (\d+|one)|'
        r'unique example|'
        r'only \d+ produced|'
        r'production number \d+'
        r'rwb|'
        r'singer|'
        r'guntherwerks|'
        r'elfwerks'
    r')\b',

    # ACCIDENT HISTORY
    'is_accident_free': r'\b('
        r'accident[- ]?free|'
        r'never crashed|'
        r'clean title|'
        r'no accidents|'
        r'undamaged'
    r')\b',

    # PERFORMANCE UPGRADES
    'has_upgrades': r'\b('
        r'KW suspension|'
        r'x51|'
        r'upgraded brakes|'
        r'recaro|'
        r'limited[- ]?slip|'
        r'aftermarket (exhaust|turbo|suspension|intake|wheels)|'
        r'performance parts|'
        r'turbo upgrade|'
        r'weissach package'
    r')\b',

    # FIRST OWNER
    'first_owner': r'\b('
        r'first owner|'
        r'one owner|'
        r'single owner|'
        r'original owner|'
        r'first hand|'
        r'single registered keeper'
    r')\b'
    }

    # Normalize text
    text_series = df[text_column].fillna("").str.lower()

    # Extract features
    for feature, pattern in patterns.items():
        df[feature] = text_series.str.contains(pattern, regex=True, flags=re.IGNORECASE).astype(int)

    # Score system (adjust weights as needed)
    score_weights = {
        'restoration_full': 2,
        'restoration_partial': 1.5,
        'is_restomod': 1.7,
        'has_docs': 0.7,
        'is_matching_numbers': 1,
        'is_mint': 0.5,
        'is_race_ready': 2.5,
        'is_rare': 2.5,
        'is_accident_free': 0.5,
        'has_upgrades': 2.3,
        'first_owner': 1.2
    }

    df['listing_score'] = df[[k for k in score_weights]].dot(pd.Series(score_weights))

    return df


In [19]:
# Calculate listing scores for all rows in clean_df
clean_df = extract_listing_features(clean_df, text_column='Description')
#clean_df = clean_excel_file("../data/listings_bronze.xlsx", output_path="../data/listings_silver.xlsx")
display(clean_df.head())

clean_df.to_excel("../data/all_listings_silver.xlsx", index=False)

C:\Users\jaxon\AppData\Local\Temp\ipykernel_21580\1492577754.py:139: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df[feature] = text_series.str.contains(pattern, regex=True, flags=re.IGNORECASE).astype(int)
C:\Users\jaxon\AppData\Local\Temp\ipykernel_21580\1492577754.py:139: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df[feature] = text_series.str.contains(pattern, regex=True, flags=re.IGNORECASE).astype(int)
C:\Users\jaxon\AppData\Local\Temp\ipykernel_21580\1492577754.py:139: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df[feature] = text_series.str.contains(pattern, regex=True, flags=re.IGNORECASE).astype(int)
C:\Users\jaxon\AppData\Local\Temp\ipykernel_21580\1492577754.py:139: UserWarning: This pattern is interpreted

,URL,level_0,index,Title,Year of construction,Model,VIN,Body,Series,Power,...,age,mileage_per_year,log_mileage,price_in_eur,model_category,model_cat_ordered,Mileage_model_cat,inv_Mileage_model_cat,Mileage_sq_model_cat,first_owner
0,https://www.elferspot.com/en/car/porsche-912-1...,2.0,2.0,Porsche 912,1967.0,912,12804201,Coupé,912,118 HP,...,58,15.517241,6.803505,179800.0,Base Carrera / Targa / 912,9,8100,0.009989,7290000,0
1,https://www.elferspot.com/en/car/porsche-911-t...,3.0,3.0,Porsche 911 T,1973.0,911 T (US),NaN,Coupé,911 F-Model,140 HP,...,52,980.769231,10.839601,82950.0,Base Carrera / Targa / 912,9,459000,0.000176,23409000000,0
2,https://www.elferspot.com/en/car/porsche-964-c...,6.0,6.0,Porsche 964 Carrera 2,1992.0,964 Carrera 2,NaN,Cabriolet,964,250 HP,...,33,4812.121212,11.975407,87500.0,Base Carrera / Targa / 912,9,1429200,0.000057,226956960000,0
3,https://www.elferspot.com/en/car/porsche-911-c...,7.0,7.0,Porsche 911 Carrera 3.2 WTL,1987.0,911 Carrera 3.2 WTL,WP0ZZZ91ZHS140870,Targa,911 G-Model,231 HP,...,38,3347.710526,11.753626,159900.0,Carrera 3.0/3.2 / S / SC,8,1017704,0.000063,129465178952,1
4,https://www.elferspot.com/en/car/porsche-993-c...,8.0,8.0,Porsche 993 Targa,1997.0,993 Carrera,NaN,Targa,993,286 HP,...,28,5265.928571,11.901224,110000.0,Base Carrera / Targa / 912,9,1327014,0.000061,195662906244,0
